In [127]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import xml.etree.ElementTree as ET
from PIL import Image, ImageEnhance
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import cv2

In [128]:
ROOT_PATH = "/Users/huylv/Desktop/Master CS Messina/Year 2/ATDA/Steel Defect Detection/Example/data/NEU-DET"
IMAGES_PATH = "/Users/huylv/Desktop/Master CS Messina/Year 2/ATDA/Steel Defect Detection/Example/data/NEU-DET/IMAGES"
ANNOTATIONS_PATH = "/Users/huylv/Desktop/Master CS Messina/Year 2/ATDA/Steel Defect Detection/Example/data/NEU-DET/ANNOTATIONS"
IMAGES_DIR = os.listdir(IMAGES_PATH)
ANNOTATIONS_DIR = os.listdir(ANNOTATIONS_PATH)
images_arr = []
annotations_arr = []
for image in IMAGES_DIR:
    images_arr.append(IMAGES_PATH + "/" + image)
for annotation in ANNOTATIONS_DIR:
    annotations_arr.append(ANNOTATIONS_PATH + "/" + annotation)

images_arr.sort()
annotations_arr.sort()

In [129]:
class BoundingBox:
    def __init__(self, _label, _xmin, _ymin, _xmax, _ymax):
        self.label = _label
        self.xmin = _xmin
        self.ymin = _ymin
        self.xmax = _xmax
        self.ymax = _ymax

In [130]:
class Image:
    def __init__(self, _name, _boundingBox, _height = 200, _weight = 200):
        self.name = _name
        self.boundingBox = _boundingBox
        self.height = _height
        self.weight = _weight
        
    def toString(self):
        return "name: {}\nbox: {}".format(self.name, self.boundingBox)

In [131]:
def loadingAnnotationImage(annotation):
    xmlFile = open(annotation)
    tree = ET.parse(xmlFile)
    root = tree.getroot()
    filename = root.find('filename').text
    label = filename.split("_")[0]
    boundBoxes = []
    for boundingObject in root.findall('object'):
        xmin = boundingObject.find('bndbox').find('xmin').text
        ymin = boundingObject.find('bndbox').find('ymin').text
        xmax = boundingObject.find('bndbox').find('xmax').text
        ymax = boundingObject.find('bndbox').find('ymax').text
        box = BoundingBox(label, xmin, ymin, xmax, ymax)
        boundBoxes.append(box)
    image = Image(filename, boundBoxes)
    return image

In [139]:
def loadingAnnotationImage_1(annotation):
    xmlFile = open(annotation)
    tree = ET.parse(xmlFile)
    root = tree.getroot()
    filename = root.find('filename').text
    label = filename.split("_")[0]
    boundingObject = root.findall('object')[0]
    xmin = boundingObject.find('bndbox').find('xmin').text
    ymin = boundingObject.find('bndbox').find('ymin').text
    xmax = boundingObject.find('bndbox').find('xmax').text
    ymax = boundingObject.find('bndbox').find('ymax').text
    box = BoundingBox(label, xmin, ymin, xmax, ymax)
    image = Image(filename, box)
    return image

In [140]:
image_bounded_arr = []
for annotation in annotations_arr:
    image_bounded_arr.append(loadingAnnotationImage_1(annotation))

In [151]:
data_image = []
labels_image = []
boxes_image = []
for i in range(len(image_bounded_arr)):
    image = image_bounded_arr[i]
    box = image.boundingBox
    xmin = float(box.xmin) / image.weight
    ymin = float(box.ymin) / image.height
    xmax = float(box.xmax) / image.weight
    ymax = float(box.ymax) / image.height
    
    loading_image = load_img(images_arr[i], target_size = (224, 224))
    loading_image = img_to_array(loading_image)
    data_image.append(loading_image)
    labels_image.append(box.label)
    boxes_image.append((xmin, ymin, xmax, ymax))
        
# convert the data, class labels, bounding boxes, and image paths to
# NumPy arrays, scaling the input pixel intensities from the range
# [0, 255] to [0, 1]
data_image = np.array(data_image, dtype="float32") / 255.0
labels_image = np.array(labels_image)
boxes_image = np.array(boxes_image, dtype = "float32")
images_arr = np.array(images_arr)

# perform one-hot encoding on the labels
labelBinarizer = LabelBinarizer()
labels_image = labelBinarizer.fit_transform(labels_image)

# only there are only two labels in the dataset, then we need to use
# Keras/TensorFlow's utility function as well

from tensorflow.keras.utils import to_categorical

# if len(labelBinarizer.classes_) == 6:
#     labels_image = to_categorical(labels_image)
# print(labels_image.shape)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
split = train_test_split(data_image, labels_image, boxes_image, images_arr, test_size=0.2, random_state = 42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]

print("trainImages.size: {}\ttestImages.size:{}".format(len(trainImages), len(testImages)))
print("trainLabels.size: {}\testLabels.size:{}".format(len(trainLabels), len(testLabels)))
print("trainBBoxes.size: {}\testBBoxes.size:{}".format(len(trainBBoxes), len(testBBoxes)))
print("trainPaths.size: {}\testPaths.size:{}".format(len(trainPaths), len(testPaths)))
# write the testing filenames to disk so that we can use then
# when evaluating/testing our bounding box regressor
print("[INFO] saving testing filenames...")
TEST_FILE_PATH = ROOT_PATH + "/Test.txt"
f = open(TEST_FILE_PATH, "w")
f.write("{}".format(testFilenames))
f.close()

trainImages.size: 1440	testImages.size:360
trainLabels.size: 1440	estLabels.size:360
trainBBoxes.size: 1440	estBBoxes.size:360
trainPaths.size: 1440	estPaths.size:360
[INFO] saving testing filenames...


In [153]:
LEARNING_RATE = 1e-4
NUM_EPOCHS = 20
BATCH_SIZE = 32

# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights = "imagenet", include_top = False, input_tensor = Input(shape = (224, 224, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid", name="bounding_box")(bboxHead)
# construct the model we will fine-tune for bounding box regression
# construct a second fully-connected layer head, this one to predict
# the class label
softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(len(labelBinarizer.classes_), activation="softmax", name="class_label")(softmaxHead)
# put together our model which accept an input image and then output
# bounding box coordinates and a class label
model = Model(inputs = vgg.input, outputs = (bboxHead, softmaxHead))

losses = {
    "class_label": "categorical_crossentropy",
    "bounding_box": "mean_squared_error",
}
# define a dictionary that specifies the weights per loss (both the
# class label and bounding box outputs will receive equal weight)
lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}

# model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(learning_rate = LEARNING_RATE)
model.compile(loss = losses, optimizer = opt, metrics = ["accuracy"], loss_weights = lossWeights)
model.summary()

Model: "model_26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_33[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [154]:
# construct a dictionary for our target training outputs
trainTargets = {
    "class_label": trainLabels,
    "bounding_box": trainBBoxes
}
# construct a second dictionary, this one for our target testing
# outputs
testTargets = {
    "class_label": testLabels,
    "bounding_box": testBBoxes
}

print(trainLabels.shape)
print(trainBBoxes.shape)
print(testLabels.shape)
print(testBBoxes.shape)

print(np.asarray(trainTargets).shape)

# train the network for bounding box regression and class label
# prediction
print("[INFO] training model...")
H = model.fit(trainImages, trainTargets, validation_data = (testImages, testTargets),
              batch_size = 32, epochs = 20, verbose = 1)
# serialize the model to disk
model.save(ROOT_PATH + "/model.h5", save_format="h5")
# serialize the label binarizer to disk
print("[INFO] saving label binarizer...")
f = open(ROOT_PATH + "/model.h5", "wb")
f.write(pickle.dumps(labelBinarizer))
f.close()

(1440, 6)
(1440, 4)
(360, 6)
(360, 4)
()
[INFO] training model...
Epoch 1/20
45/45 [==============================] - 355s 8s/step - loss: 1.3154 - bounding_box_loss: 0.0487 - class_label_loss: 1.2667 - bounding_box_accuracy: 0.6812 - class_label_accuracy: 0.5153 - val_loss: 0.5745 - val_bounding_box_loss: 0.0408 - val_class_label_loss: 0.5337 - val_bounding_box_accuracy: 0.7389 - val_class_label_accuracy: 0.8861
Epoch 2/20
 4/45 [=>............................] - ETA: 2:55 - loss: 0.6948 - bounding_box_loss: 0.0430 - class_label_loss: 0.6518 - bounding_box_accuracy: 0.7656 - class_label_accuracy: 0.7812

KeyboardInterrupt: 

In [ ]:
LEARNING_RATE = 1e-4
NUM_EPOCHS = 25
BATCH_SIZE = 32
opt = Adam(lr = LEARNING_RATE, decay = LEARNING_RATE / NUM_EPOCHS)

baseModel = tf.keras.applications.ResNet152V2(weights='imagenet')
headModel = baseModel.output
headModel = AveragePooling2D(pool_size = (7, 7))(headModel)
headModel = Flatten(name = "flatten")(headModel)
headModel = Dense(256, activation = "relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(6, activation = "softmax")(headModel)
model = Model(inputs = baseModel.input, outputs = headModel)

for layer in baseModel.layers:
    layer.trainable = False
    
model.compile(loss = "sparse_categorical_crossentropy", optimizer = opt, metrics = ['accuracy'])

history = model.fit(trainImages, trainTargets, validation_data = (testImages, testTargets), epochs = NUM_EPOCHS)
# H = model.fit_generator(
# 	trainGen,
# 	steps_per_epoch=totalTrain // config.BS,
# 	validation_data=valGen,
# 	validation_steps=totalVal // config.BS,
# 	epochs=config.NUM_EPOCHS)
# fit(
#     x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None,
#     validation_split=0.0, validation_data=None, shuffle=True, class_weight=None,
#     sample_weight=None, initial_epoch=0, steps_per_epoch=None,
#     validation_steps=None, validation_batch_size=None, validation_freq=1,
#     max_queue_size=10, workers=1, use_multiprocessing=False
# )

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised = True, with_info = True)
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

(train_set, valid_set, test_set), dataset_info = tfds.load(
    'tf_flowers',
    split=['train[:70%]', 'train[:15%]', 'train[:10%]'],
    with_info = True,
    as_supervised=True,
)
print(dataset_size)
print(class_names)
print(n_classes)